<a href="https://colab.research.google.com/github/MatinAminSabouri/Food_RAG_System/blob/main/Food_Rag_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00


In [1]:
from PIL import Image
import requests
import torch
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [2]:
from google.colab import files
uploaded = files.upload()  # opens a file picker

image = Image.open("sushi.jpg")
image.show()

# Define possible food labels (just for my MVP)
food_labels = ["Spaghetti Carbonara", "Hamburger", "Sushi", "Caesar Salad", "Chicken Tikka",
               "Pad Thai", "Pho", "Ramen", "Pizza Margherita", "Tacos"]


Saving sushi.jpg to sushi.jpg


In [3]:
# Prepare inputs
inputs = processor(text=food_labels, images=image, return_tensors="pt", padding=True)


In [4]:
# Predict
with torch.no_grad():
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)

# Get prediction
predicted_index = torch.argmax(probs).item()
predicted_label = food_labels[predicted_index]
#print(f"🍽️ Predicted dish: {predicted_label}")


In [14]:
import os

os.environ["SPOONACULAR_API_KEY"] = "YOur Api key"


In [11]:
import os
import requests

API_KEY = os.environ.get("SPOONACULAR_API_KEY")

if not API_KEY:
    raise ValueError("API key not found. Please set the SPOONACULAR_API_KEY environment variable.")


In [12]:
def get_recipe(dish_name):
    search_url = "https://api.spoonacular.com/recipes/complexSearch"
    params = {
        "query": dish_name,
        "number": 1,
        "apiKey": API_KEY
    }

    #Search food
    response = requests.get(search_url, params=params)
    data = response.json()

    if not data["results"]:
        return "No Result!"

    recipe_id = data["results"][0]["id"]
    title = data["results"][0]["title"]

    info_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
    info_params = {
        "includeNutrition": False,
        "apiKey": API_KEY
    }
    info_response = requests.get(info_url, params=info_params)
    info_data = info_response.json()

    ingredients = [ing["original"] for ing in info_data["extendedIngredients"]]
    instructions = info_data.get("instructions", "No instructions provided.")

    return {
        "title": title,
        "ingredients": ingredients,
        "instructions": instructions
    }


In [13]:
print(f"🍽️ Predicted dish: {predicted_label}")

recipe = get_recipe(predicted_label)

print("\n🍽️ Title:", recipe["title"])
print("\n🧂 Ingredients:")
for i, ing in enumerate(recipe["ingredients"], 1):
    print(f"{i}. {ing}")

print("\n👨‍🍳 Instructions:\n", recipe["instructions"])


🍽️ Predicted dish: Sushi

🍽️ Title: Japanese Sushi

🧂 Ingredients:
1. Cooked octopus
2. Cooked prawns
3. Raw tuna
4. Salmon
5. Salmon caviar
6. Japanese sticky rice
7. Lava seaweed
8. Wasabi
9. Asparagus
10. Shiitake mushrooms

👨‍🍳 Instructions:
 Recipe one: Pour cooked rice into a tray. Finely slice the salmon and the tuna.
Put the seaweed onto a sushi mat and press rice down onto it.
Slice the mushrooms and asparagus pieces and lay along the rice. Add the wasabi. Roll the sushi and cut into slices. Serve with salmon caviar, wasabi and soy.
Recipe two: Squeeze the rice into balls and press the prawns, tuna or salmon into the top.
